### Trying different ways to produce binary strings for Code-VEP

In [44]:
from itertools import islice
from plotly import graph_objects as go
import numpy as np
from typing import List


def get_data_np(nbits=32, ncodes=4) -> List[List[int]]:
    res = []
    np.random.seed(1)
    for _ in range(ncodes):
        res.append(np.random.randint(2, size=nbits))
    return res

# These positions result in LFSR that continues for longest possible time without repeating
# Taken from https://docs.rs/lfsr/0.2.0/lfsr/galois/index.html
set_positions = [
    (2, 1),
    (3, 2),
    (4, 3),
    (5, 3),
    (6, 5),
    (7, 6),
    (8, 6, 5, 4),
    (9, 5),
    (10, 7),
    (11, 9),
    (12, 11, 8, 6),
    (13, 12, 10, 9),
    (14, 13, 11, 9),
    (15, 14),
    (16, 14, 13, 11),
    (17, 14),
    (18, 11),
    (19, 18, 17, 14),
    (20, 17),
    (21, 19),
    (22, 21),
    (23, 18),
    (24, 23, 21, 20),
    (25, 22),
    (26, 25, 24, 20),
    (27, 26, 25, 22),
    (28, 25),
    (29, 27),
    (30, 29, 26, 24),
    (31, 28),
    (32, 30, 26, 25),
]
polynoms = {}
for idx, s_tup in enumerate(set_positions):
    tmp = 0 << 31
    for s in s_tup:
        tmp = tmp | (1 << (s - 1))
    polynoms[idx] = bin(tmp)

def lfsr(nbits):
    data = 1234
    poly = polynoms.get(nbits, 0b1)
    while True:
        lsb = data & 1
        data = data >> 1
        if lsb != 0:
            data = data ^ poly
            yield 1
        else:
            yield 0


def get_data_lfsr(nbits=32, ncodes=4):
    """LFSR needs a starting binary string, and needs to know which bits to flip during each iteration.
    Choosing the right bits to flip is a tough (previously solved) problem - the table above includes the best values
    for working with strings between length 1 and length 32. However - not sure what the starting binary strings should be,
    and this seems to also be important. Starting with all 0 definitely does not work"""
    x = lfsr(nbits)
    res = []
    for _ in range(ncodes):
        res.append(np.array(list(islice(x, nbits))))
    return res


{0: '0b11', 1: '0b110', 2: '0b1100', 3: '0b10100', 4: '0b110000', 5: '0b1100000', 6: '0b10111000', 7: '0b100010000', 8: '0b1001000000', 9: '0b10100000000', 10: '0b110010100000', 11: '0b1101100000000', 12: '0b11010100000000', 13: '0b110000000000000', 14: '0b1011010000000000', 15: '0b10010000000000000', 16: '0b100000010000000000', 17: '0b1110010000000000000', 18: '0b10010000000000000000', 19: '0b101000000000000000000', 20: '0b1100000000000000000000', 21: '0b10000100000000000000000', 22: '0b110110000000000000000000', 23: '0b1001000000000000000000000', 24: '0b11100010000000000000000000', 25: '0b111001000000000000000000000', 26: '0b1001000000000000000000000000', 27: '0b10100000000000000000000000000', 28: '0b110010100000000000000000000000', 29: '0b1001000000000000000000000000000', 30: '0b10100011000000000000000000000000'}


In [41]:
res1 = get_data_np(32, 4)
res2 = get_data_lfsr(32, 4)

In [42]:
fig = go.Figure()
for i in range(4):
    fig.add_trace(go.Scatter(x=np.arange(len(res1[i])), y=2*i + res1[i]))
fig.show()


In [43]:
fig = go.Figure()
for i in range(4):
    fig.add_trace(go.Scatter(x=np.arange(len(res1[i])), y=2*i + res2[i]))
fig.show()
